In [ ]:
# Import the requrired librarires
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time,os
from tqdm import tqdm_notebook

In [ ]:
# Enter the file name to get the links from and the file name to save the scraped descriptions
# Modify this cell to download from various platforms
merged_df = pd.read_csv('datasets/platform_url_merged/switch_merged.csv')
save_file_name = 'switch_description_test'
# Go up a directory
%cd ..

In [ ]:
# Have a peek at the source file
merged_df = merged_df.head(10)
merged_df

In [ ]:
# Funciton to scrape the details from each link
def get_data(driver,title,url):
    driver.get(url)
    try:
        name = driver.find_element(By.CSS_SELECTOR,'div.product_title > a:nth-child(1) > h1:nth-child(1)')
        title = name.text
    except:
        title = 'N/A'
    try:
        toggle = driver.find_element(By.CSS_SELECTOR,'.product_summary > span:nth-child(2) > span:nth-child(1) > span:nth-child(4)')
        if toggle.text == "Expand":
            toggle.click()
    except:
        pass
    try:
        summary = driver.find_element(By.CSS_SELECTOR,'.product_summary > span:nth-child(2) > span:nth-child(1) > span:nth-child(2)')
        description = summary.text
    except:
        try:
            summary = driver.find_element(By.CSS_SELECTOR,'.product_summary > span:nth-child(2) > span:nth-child(1)')
            description = summary.text
        except:
            description = 'N/A'
    try:
        genre_list = driver.find_element(By.CSS_SELECTOR,'.product_genre')
        genre = genre_list.text
    except:
        genre = 'N/A'
    return title,description,genre


In [ ]:
# driver = webdriver.Firefox()
# Initialize the webdriver headlessly for faster scraping
options = webdriver.FirefoxOptions()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.get('https://www.metacritic.com/game/xbox-one/ea-sports-ufc')

In [ ]:
# Check to see if any previous file has been saved to continue from there
start_index = 0
platform_data = []
try:
    # Try to read the previously saved csv file
    df = pd.read_csv(f"datasets/platform_description_merged/{save_file_name}.csv")
    # Get the index of the last row
    start_index = df.index[-1] + 1
    platform_data = df.to_dict('records')
except FileNotFoundError:
    # If the file doesn't exist, start from the first row
    start_index = 0

print('starting from index = ' + str(start_index))

In [ ]:
# Start the scraping
start = time.time()
# iterate through all the rows
for index, row in tqdm_notebook(merged_df.iloc[start_index:].iterrows(), total=merged_df.shape[0] - start_index):
    url = row['url']
    title = row['title']
    # do your processing with the URL
    title, description, genre = get_data(driver,title,url)
    platform_data.append({
        'title': title,
        'description': description,
        'genres': genre
    })

    # Save the data to the csv file after each iteration
    df = pd.DataFrame(data=platform_data, columns=platform_data[0].keys())
    df.to_csv(f"datasets/platform_description_merged/{save_file_name}.csv", index=False)

end = time.time()
print(end - start)
driver.close()